In [2]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_quat=melb_df['Date'].dt.quarter.value_counts()
#display(melb_quat)
melb_quat.iloc[1]
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # 
        
melb_df.sort_values(by='Price').head(10)     
melb_df.sort_values(by='AreaRatio', ascending=False, ignore_index=True).iloc[15]

#melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

melb_df[(melb_df['Type']=='townhouse')& (melb_df['Rooms']>2)].sort_values(
    by=['Rooms','MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).iloc[18]

melb_df.groupby(by='Type').mean(numeric_only=True)
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)
melb_df.groupby('MonthSale')['Price'].agg('describe')
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)
melb_df.groupby('Rooms')['Price'].mean(numeric_only=True)  
melb_df.groupby('Regionname')['Lattitude'].std().sort_values(ascending=False)     
d1=pd.to_datetime('2017-05-01')
d2=pd.to_datetime('2017-09-01')
melb_df[(melb_df['Date']>d1)&(melb_df['Date']<d2)].groupby('SellerG')['Price'].sum().sort_values(ascending=False)

melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)
                                 
pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
    fill_value=0
)


                              

In [3]:
dates=pd.read_csv('data/dates.csv')
movies=pd.read_csv('data/movies.csv')
ratings1=pd.read_csv('data/ratings1.csv')
ratings2=pd.read_csv('data/ratings2.csv')

movies.nunique()
ratings1.nunique()
dates['date']=pd.to_datetime(dates['date'])
dates['date'].dt.year.mode()

ratings = pd.concat([ratings1, ratings2], ignore_index=True)
ratings = ratings.drop_duplicates(ignore_index=True)
ratings_dates = pd.concat([ratings, dates], axis=1)

joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)

In [30]:
ratings_movies=pd.read_csv('data/ratings_movies.csv')

#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    
ratings_movies['year_relise']=ratings_movies['title'].apply(get_year_release)  
mask=(ratings_movies['year_relise']==2018)
ratings_movies.groupby('userId')['genres'].nunique().sort_values()
ratings_movies.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])  
mask1=(ratings_movies['rating'].count()>10)
ratings_movies[mask].groupby('genres')['rating'].agg(['count', 'mean']).sort_values(['count', 'mean'])

ratings_movies['date']=pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating']=ratings_movies['date'].dt.year
new_table=ratings_movies.pivot_table(values='rating', index='year_rating', columns='genres')
max_new=new_table.loc[2018].sort_values(ascending=False)
max_new['Animation|Children|Mystery']
new_table['Action|Adventure|Animation|Children|Comedy|IMAX']

year_rating
1996         NaN
1997         NaN
1998         NaN
1999         NaN
2000         NaN
2001         NaN
2002         NaN
2003         NaN
2004         NaN
2005         NaN
2006         NaN
2007         NaN
2008         NaN
2009    3.250000
2010    3.500000
2011         NaN
2012    3.550000
2013    5.000000
2014    3.500000
2015    3.375000
2016    3.666667
2017    1.000000
2018    3.700000
Name: Action|Adventure|Animation|Children|Comedy|IMAX, dtype: float64

In [48]:
orders=pd.read_csv('data/orders.csv', sep=';')
products=pd.read_csv('data/products.csv', sep=';')
orders_products=orders.merge(products, left_on='ID товара', right_on='Product_ID',how='left')


orders_products=orders_products.loc[orders_products['Product_ID']!='NaN']

orders_products[orders_products['Отменен'] == 'Да']['Name']
orders_products['profit']=orders_products['Количество']*orders_products['Price']
#display(orders_products)
mask=orders_products['Оплачен']=='Да'
orders_products[mask].groupby('ID Покупателя')['profit'].sum()



ID Покупателя
1        0.0
5    13043.0
7    17096.0
8     1344.0
Name: profit, dtype: float64